In [101]:
import numpy as np
from numpy import array, sin, cos, dot, random, zeros, ones
from scipy.optimize import minimize, root
from scipy.linalg import solve, norm
from math import pi
import matplotlib.pyplot as plt
from matplotlib import animation,rc
%matplotlib inline
from IPython.display import HTML

In [79]:
def acrobot_motion(x, u):
    friction = 0.1
    g = 9.8
    M = array([[3 + 2*cos(x[1]), 1+cos(x[1])],[1+cos(x[1]), 1]])
    c1 = x[3]*(2*x[2]+x[3])*sin(x[1]) + 2*g*sin(x[0]) + g*sin(x[0]+x[1])
    c2 = -x[2]**2*sin(x[1]) + g*sin(x[0]+x[1])
    a = np.concatenate([x[2:4], solve(M,array([c1-friction*x[2], c2-friction*x[3]]))])
    B = np.concatenate([[0,0],solve(M,array([0,1]))])
    xd = a + B*u
    return xd
r=k=1
cost_function = lambda x,u,r,k : r/2*u**2 + 1 - exp(k*cos(x[0]) + k*cos(x[1])-2*k)
cost_function = lambda x,u : u**2 + norm(x,x)
n=100
dt=0.1

#cost_function = lambda x,u : u**2

def lagrangian(input_array):
    #x is x1 to xN,followed by U0 to UN-1, followed by lambda0 to lambdaN-1
    x = np.concatenate([[pi,0,0,0],input_array[:(n-1)*4]])
    u = input_array[(n-1)*4:(n-1)*4+n]
    lam = input_array[(n-1)*4+n:-1]

    L=0
    for i in range(n-1):
        #current x
        xx = x[i*4:(i+1)*4]
        #next x
        xx_p1 = x[(i+1)*4:(i+2)*4]
        #difference between x+dt*dx and x_next
        delta = xx+dt*acrobot_motion(xx,u[i])-xx_p1
        #minimize energy
        cost = cost_function(xx,u[i])
        lagragian_term = lam[i]*(delta[0:1]%(2*pi))
        L+=cost+lagragian_term
    return L
init = np.ones(n*4+n+n)
out = minimize(lagrangian,init)

In [47]:
# plot


array([        0.        , -26650117.91403154,         0.        ,
               0.        ])

array([ -8.59273787e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,  -1.74317385e+00,  -1.74317385e+00,
        -1.74317385e+00,

In [80]:
out


      fun: -1135663995.869996
 hess_inv: array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])
      jac: array([  3.61493072e+08,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,

In [20]:
M

array([[ 2.16770633,  0.58385316],
       [ 0.58385316,  1.        ]])

In [102]:
def acrobot_next_step(x,u):
    xd = acrobot_motion(x,u)
    new_x = x + xd*dt
    #new_x[0]%=(2*pi)
    #new_x[1]%=(2*pi)
    return new_x

def grad_lagrangian_trick(u):
    # u is a vector of u0 to uN-1
    n=len(u)
    x = zeros([n+1,4])
    x[0]=array([pi,0,0,0])
    lam = zeros([n,4])
    
    lam[n-1] = zeros(4)
    out = zeros(n)
    dgdx = lambda x,u : 2*abs(x)
    dgdu = lambda x,u : 2*u
    
    delta = 0.0001
    def dfdx(x,u):
        out = zeros([4,4])
        for i in range(4):
            delta_v = zeros(4)
            delta_v[i] = delta
            #print(x+delta_v)
            out[i] = (acrobot_next_step(x+delta_v,u)-acrobot_next_step(x-delta_v,u))/delta
        return out
    dfdu = lambda x,u : (acrobot_next_step(x,u+delta)-acrobot_next_step(x,u-delta))/delta
    
    for i in range(n):
        #print (x[i])
        x[i+1]=acrobot_next_step(x[i],u[i])
        x[i+1][0]%=(2*pi)
        x[i+1][1]%=(2*pi)
        
    for i in reversed(range(1,n)):
        #print((x[i]))
        #print(dfdx(x[i],u[i]))
        #print(lam[i])
        lam[i-1]=dgdx(x[i],u[i])+dot(dfdx(x[i],u[i]),lam[i])
    
    for i in range(n):
        out[i]=dgdu(x[i],u[i])+dot(lam[i],dfdu(x[i],u[i]))
    return out
u = ones(n)
out = root(grad_lagrangian_trick,u)

In [99]:
out

    fjac: array([[ -6.08232913e-01,  -7.45972584e-01,  -1.08994974e-01, ...,
         -2.04293036e-17,   5.03778733e-20,   0.00000000e+00],
       [ -7.16265552e-01,   4.73684673e-01,   1.23210543e-01, ...,
          1.07945220e-17,  -2.06833691e-21,  -0.00000000e+00],
       [  2.35729354e-01,  -2.00901746e-01,  -4.30918403e-01, ...,
          2.85952261e-17,  -4.05853862e-20,   0.00000000e+00],
       ..., 
       [ -1.23371581e-26,   2.09237485e-26,  -1.99317407e-25, ...,
         -9.16232169e-01,   2.94938755e-01,   0.00000000e+00],
       [  9.97537415e-29,   2.94870099e-29,   7.27055627e-28, ...,
         -2.67276570e-01,  -9.51509222e-01,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,  -1.00000000e+00]])
     fun: array([  1.18231399e+35,   5.81157827e+34,   2.10484359e+34,
         4.62747816e+33,  -6.02899094e+32,  -1.36502873e+33,
        -9.20250196e+32,  -4.32633901e+32,  -1.48837928e+32,
   